<a href="https://colab.research.google.com/github/somosnlp/recursos/blob/main/hackathon_2024/creacion_de_datasets_sinteticos_con_distilabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agradecimientos

Muchas gracias al equipo de Argilla por preparar este notebook de ejemplo, en especial a Daniel Vila Suero (CEO y fundador) y Agustín Piqueres (MLE).

Muchas gracias también a Hugging Face por darnos la oportunidad de disfrutar de la PRO API durante el hackathon. Pedimos a todos los equipos responsabilidad, por favor utilizad esta API para el desarrollo de proyectos del hackathon. Así seguiremos pudiendo organizar estos maravillosos eventos gratuitos. ¡Gracias!

# Instalar requisitos

In [ ]:
%pip install -U distilabel[hf-inference-endpoints,argilla] -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.3/254.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.8 MB/s eta 0:00:00


In [ ]:
from distilabel.llm.huggingface.inference_endpoints import InferenceEndpointsLLM
from distilabel.tasks import TextGenerationTask
from distilabel.tasks import SelfInstructTask
from distilabel.pipeline import Pipeline

In [ ]:
import distilabel
distilabel.__version__

'0.6.0'

# Introducción

En este tutorial se muestra como generar conjuntos de datos sintéticos en Español para entrenar y mejorar modelos del lenguaje en Español.

Para ello se utiliza `distilabel` de Argilla, una librería escalable para generar datasets para LLMs.

Este cuaderno provee una breve guía de introducción pero se recomienda leer la [documentación](https://distilabel.argilla.io/latest/) y explorar opciones más avanzadas así como casos de uso interesantes, más allá del ejemplo utilizado aquí.

En este cuaderno:

- Se muestra como generar instrucciones y respuestas para SFT (supervised fine tuning) utilizando Hugging Face Inference for PRO (gracias al sponsorship de Hugging Face).

- Se muestra como generar instrucciones y respuestas para SFT (supervised fine tuning) utilizando la GPU de Colab y modelos locales.


# Generación de instrucciones con HF Inference endpoints

Con este apartado, los equipos pueden generar instrucciones en Español sobre distintos temas y para distintas aplicaciones. Aquí se muestra solo un ejemplo muy básico.

Para ejecutar este apartado es necesario formar parte de la organización SomosNLP en Hugging Face y configurar el token personal (nivel write) para poder hacer uso de la cuenta PRO.


Se ruega no sobrecargar la API de inferencia y hacer pruebas con pequeñas muestras hasta tener claro el caso de uso y en cualquier caso no generar datasets de más de 5000 ejemplos.

## Comprobar acceso a Inference Endpoints

In [ ]:
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')

# change endpoint name and namespace once deployed
ENDPOINT_NAME = "mistralai/Mixtral-8x7B-Instruct-v0.1"


llm = InferenceEndpointsLLM(
    endpoint_name_or_model_id=ENDPOINT_NAME,
    task=TextGenerationTask(),
    token=hf_token,
    prompt_format="llama2"
)

INFO:distilabel:Using Serverless Inference Endpoint


In [ ]:
llm.generate([{"input": "Generate a random joke in Spanish, just the joke, no greetings"}])

[[{'model_name': 'mistralai/Mixtral-8x7B-Instruct-v0.1',
   'prompt_used': "<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>\n\nGenerate a random joke in Spanish, just the joke, no greetings [/INST]",
   'raw_output': ' ¿Por qué el pollo siempre cruza la calle?\nPorque quiere llegar al otro lado y decir: "¡Adivinen quién acaba de cruzar la calle!" (Why does the chicken always cross the road? Because it wants to get to the other side and say: "Guess who just crossed the road!")',
   'parsed_output': {'generati

## Generar dataset de instrucciones por temas

Esto es solo un ejemplo y configurando la lista de `topics` y la `application_description` se pueden generar instrucciones de mucho tipos y dominios, sé creativ@!

In [ ]:
from datasets import Dataset

topics = [
  "Matemáticas",
  "Física",
  "Química",
  "Biología",
  "Informática",
  "Ingeniería",
  "Astronomía",
  "Geología",
  "Ciencias Ambientales",
  "Robótica",
  "Estadística",
  "Ciencias de Materiales",
  "Nanotecnología",
  "Genética",
  "Oceanografía",
  "Meteorología",
  "Farmacología",
  "Neurociencia",
  "Bioquímica",
  "Física de Partículas",
  "Ciencia de Datos",
  "Inteligencia Artificial",
  "Sostenibilidad",
  "Energías Renovables"
]


dataset = Dataset.from_dict({
    "input": topics
})

Te invitamos a probar diferentes prompts y ver cuál da mejores resultados:

In [ ]:
application_description = (
    "An AI assistant adept at answering a wide array of math, logic, and reasoning puzzles, trivia, "
    "and general questions. Users of this assistant love to ask the assistant to think and outlines "
    "the solutions step by step. It expects complete questions from users providing all the details "
    "to solve the proposed problem or respond to general knowledge questions. It covers general "
    "knowledge about math, puzzles, reasoning exercises, and real-life scenarios where math and "
    "reasoning are important. Highly important!! You can only generate text in SPANISH"
)

# Por defecto, `SelfInstructTask` generará 5 instrucciones pero se puede modificar este comportamiento con el argumento `num_instructions`.
instruction_task = SelfInstructTask(
    application_description=application_description
)

print(f"`SelfInstructTask`\n   - Input arguments: {instruction_task.input_args_names}\n   - Output arguments: {instruction_task.output_args_names}")

`SelfInstructTask`
   - Input arguments: ['input']
   - Output arguments: ['instructions']


In [ ]:
llm = InferenceEndpointsLLM(
    endpoint_name_or_model_id=ENDPOINT_NAME,
    task=instruction_task,
    token=hf_token,
    prompt_format="llama2",
    num_threads=4
)

pipeline = Pipeline(generator=llm)
distiset = pipeline.generate(
    dataset=dataset,
    num_generations=5,
    batch_size=4,
    display_progress_bar=True
)

In [ ]:
distiset.to_pandas().head(5)

,input,generation_model,generation_prompt,raw_generation_responses,instructions
0,Matemáticas,"[mistralai/Mixtral-8x7B-Instruct-v0.1, mistral...",[<s>[INST] <<SYS>>\nYou are an expert prompt w...,"[ 1. ""Explica detalladamente cómo calcular la ...","[[""Explica detalladamente cómo calcular la raí..."
1,Física,"[mistralai/Mixtral-8x7B-Instruct-v0.1, mistral...",[<s>[INST] <<SYS>>\nYou are an expert prompt w...,"[ 1. ""Explica paso a paso cómo funciona la seg...","[[""Explica paso a paso cómo funciona la segund..."


### Inspeccionar el dataset en argilla

A continuación vamos a crear un espacio en argilla para poder inspeccionar las instrucciones generadas en nuestra pipeline. Podemos crear una instancia de argilla como un espacio de HuggingFace. A continuación se ofrece un ejemplo para hacerlo utilizando la librería de `huggingface_hub`.

In [ ]:
rg_distiset = distiset.to_argilla(vector_strategy=False, metric_strategy=False)

In [ ]:
from huggingface_hub import duplicate_space

# Crea un HF Space de argilla programáticamente
from_id = "argilla/argilla-template-space"
# Recuerda actualizar esta variable con el nombre del dataset
dataset_name = "mi-dataset"
to_id = f"{dataset_name}-distiset"
new_space = duplicate_space(from_id, to_id=to_id)
new_space

Esto puede llevar unos minutos, puedes visitar el espacio accediendo a `new_space.url`. Una vez que esté listo, el usuario para acceder y contraseña son los que vienen por defecto

- usuario: `argilla`
- contraseña: `12345678`

A continuación nos conectamos a nuestra instancia para poder subir el dataset:

In [ ]:
import argilla as rg

argilla_api_key = "admin.apikey"
argilla_space_url = f"https://{new_space.namespace}-{to_id}.hf.space"

workspace = "admin"

rg.init(
    api_key=argilla_api_key,
    api_url=argilla_space_url,
    workspace=workspace
)

Y estamos listos para subir nuestro dataset con las instrucciones para revisarlas antes de avanzar al siguiente paso. Para ello transformamos nuestro dataset al formato necesario de argilla utilizando `to_argilla`, y subimos el dataset a la instancia de argilla utilizando `push_to_argilla`.

In [ ]:
rg_distiset = distiset.to_argilla(vector_strategy=False, metric_strategy=False)

In [ ]:
rg_distiset.push_to_argilla(name="instrucciones-distiset", workspace=workspace)

### Transformar a un dataset para generación de respuestas

A continuación vamos a transformar nuestro dataset con instrucciones al formato esperado por `distilabel` para la generación, extrayendo todas las instrucciones anidadas, y poniendo la columna "input".

In [ ]:
rows = []
from datasets import Dataset

generations = []
for row in distiset:
    for instructions in row["instructions"]:
        for generation in instructions:
            generations.append(generation)

generation_dataset = Dataset.from_dict({"input": generations})

In [ ]:
generation_dataset

Dataset({
    features: ['input'],
    num_rows: 14
})

# Generación de respuestas con HF Inference endpoints

En esta sección vamos a utilizar el dataset previo `generation_dataset` para generar conjuntos de instrucciones y problemas sintéticos para poder poder ajustar nuestro propio modelo utilizando Supervised Fine Tuning (SFT):

Vamos a crear una tarea genérica para generación de texto, reutilizando la misma descripción que pasamos a nuestra tarea previa a modo de *system_prompt*, para guiar al modelo:

In [ ]:
text_generation_task = TextGenerationTask(system_prompt=application_description)
text_generation_task

TextGenerationTask(system_prompt='An AI assistant adept at answering a wide array of math, logic, and reasoning puzzles, trivia, and general questions. Users of this assistant love to ask the assistant to think and outlines the solutions step by step. It expects complete questions from users providing all the details to solve the proposed problem or respond to general knowledge questions. It covers general knowledge about math, puzzles, reasoning exercises, and real-life scenarios where math and reasoning are important. Highly important!! You can only generate text in SPANISH', principles_distribution=None)

La forma de llamar a nuestra `pipeline` va a ser muy similar en este caso, tendremos una nueva `Task`, y dado que los problemas pueden requerir mayor cantidad de texto, vamos a modificar `max_new_tokens` a 1024.

In [ ]:
llm = InferenceEndpointsLLM(
    endpoint_name_or_model_id=ENDPOINT_NAME,
    task=text_generation_task,
    token=hf_token,
    prompt_format="llama2",
    max_new_tokens=1024,
    num_threads=4
)

pipeline = Pipeline(generator=llm)
distiset_generations = pipeline.generate(
    dataset=generation_dataset,
    num_generations=1,
    batch_size=8,
)

Igual que hicimos con nuestras instrucciones, vamos a subir ahora nuestro dataset junto con las respuestas generadas para ver lo que ha generado nuestro modelo.

In [ ]:
rg_distiset_generations = distiset_generations.to_argilla(vector_strategy=False, metric_strategy=False)

In [ ]:
rg_distiset_generations.push_to_argilla(name="sciency-distiset", workspace=workspace)

# Push al hub

Una vez esté el dataset listo, súbelo a la organización de [SomosNLP](https://huggingface.co/organizations/somosnlp/share/qgytUhPKvxVxsbZWTzVUAUSUnZmVXNPmjc) del hub de Hugging Face. ¡Este paso es imprescindible para participar en el hackathon!